In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
dbutils.widgets.text("env","")
dbutils.widgets.text("client_id","")
dbutils.widgets.text("tax_year","")
env=dbutils.widgets.get("env")
tax_year=dbutils.widgets.get("tax_year")
client_id=dbutils.widgets.get("client_id")

In [0]:
import sys
sys.path.append("/Workspace/Users/mayur10594@gmail.com/ETL_project")

In [0]:
from config.utils import get_logger
logger = get_logger("dataloadinitiate")

In [0]:
import json, yaml, os
from config.utils import load_config,create_tables
from schemas import user_schema,trans_schema,cards_schema

In [0]:
path="/Workspace/Users/mayur10594@gmail.com/ETL_project/config/etl_main.yaml"
env_config,tables_config=load_config(path,env)

In [0]:
in_path=env_config['incoming_path']
logger.info(f"Loading data from {in_path}")
logger.info(f"env is set to {env}")
logger.info(f"tax_year is set to {tax_year}")
logger.info(f"client_id is set to {client_id}")

In [0]:
schemas={
    "transactions": trans_schema,
    "users": user_schema,
    "cards": cards_schema
}

In [0]:
create_tables(spark,schemas,'dbfs')

In [0]:
df_trans=spark.read.csv(f"{in_path}/transactions_data.csv",schema=trans_schema,header=True)
df_users=spark.read.csv(f"{in_path}/users_data.csv",schema=user_schema,header=True)
df_cards=spark.read.csv(f"{in_path}/cards_data.csv",schema=cards_schema,header=True)

In [0]:
df_cards=df_cards.withColumn('create_date',current_date()).withColumn('create_user',lit('cpprod'))
df_trans=df_trans.withColumn('create_date',current_date()).withColumn('create_user',lit('cpprod'))
df_users=df_users.withColumn('create_date',current_date()).withColumn('create_user',lit('cpprod'))

In [0]:
df_users.write.format("delta").options(mergeSchema=True).mode('overwrite').saveAsTable(f"dbfs.{client_id}_users")
df_cards.write.format("delta").options(mergeSchema=True).mode('overwrite').saveAsTable(f"dbfs.{client_id}_cards")
df_trans.write.format("delta").options(mergeSchema=True).mode('overwrite').saveAsTable(f"dbfs.{client_id}_{tax_year}_transactions")


In [0]:
users_insert_count=df_users.count()
trans_insert_count=df_trans.count()
cards_insert_count=df_cards.count()


In [0]:
dbutils.notebook.exit(json.dumps({
    "users_insert_count": users_insert_count,
    "trans_insert_count": trans_insert_count,
    "cards_insert_count": cards_insert_count
}))

In [0]:

logger.info(f'''
    "users_insert_count": users_insert_count,
    "trans_insert_count": trans_insert_count,
    "cards_insert_count": cards_insert_count"
    ''')
logger.info(f"exiting the dataloadinitiate notebook")